# New Section

## Imports and Dependencies

In [ ]:
! pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
! pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
! pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
from google.colab import userdata
import os
from neo4j import GraphDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import re
import json

## Setup Neo4j Connection

In [ ]:
uri = userdata.get('NEO4J_URI')
username = userdata.get('NEO4J_USERNAME')
password = userdata.get('NEO4J_PASSWORD')
database = userdata.get('NEO4J_DATABASE')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY
GPLACES_API_KEY = userdata.get('GPLACES_API_KEY')
os.environ['GPLACES_API_KEY']=GPLACES_API_KEY

In [ ]:
class App:
    def __init__(self, uri, user, password, database=None):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database=database)
        self.database = database

    def close(self):
        self.driver.close()

    def query(self, query):
        return self.driver.execute_query(query)

    def query_params(self, query, parameters):
        return self.driver.execute_query(query, parameters_=parameters)

    def count_nodes_in_db(self):
        query = "MATCH (n) RETURN COUNT(n)"
        result = self.query(query)
        (key, value) = result.records[0].items()[0]
        return value

    def remove_nodes_relationships(self):
        query ="""
            CALL apoc.periodic.iterate(
                "MATCH (c) RETURN c",
                "WITH c DETACH DELETE c",
                {batchSize: 1000}
            )
        """
        result = self.query(query)

    def remove_all_constraints(self):
        query ="""
            CALL apoc.schema.assert({}, {})
        """
        result = self.query(query)

In [ ]:
app = App(uri, username, password, database)

In [ ]:
app.count_nodes_in_db()

7737

## Refactoring

### Correct Beer Brands names

In [ ]:
query = """
  MATCH (b:BeerBrand)
  RETURN b.name
"""

In [ ]:
result = app.query(query)

In [ ]:
beerbrands = [r.values()[0] for r in result.records]
brands_df = pd.DataFrame(data=beerbrands, columns=['beer_brand'])

In [ ]:
brands_df.head(10)

,beer_brand
0,3 Schténg
1,IV Saison[5][6][7][8]
2,V Cense[6][9][10][11]
3,VI Wheat[6][12][13][14]
4,7PK
5,26.2 M[15][16][17]
6,28 Brett
7,28 Pale Ale
8,28 Triple
9,44 Blanche


In [ ]:
brands_df['beer_brand_corrected'] = brands_df['beer_brand'].apply(lambda x: re.sub(r'[[0-9]*]', '', x))

In [ ]:
brands_df.head(20)

,beer_brand,beer_brand_corrected
0,3 Schténg,3 Schténg
1,IV Saison[5][6][7][8],IV Saison
2,V Cense[6][9][10][11],V Cense
3,VI Wheat[6][12][13][14],VI Wheat
4,7PK,7PK
5,26.2 M[15][16][17],26.2 M
6,28 Brett,28 Brett
7,28 Pale Ale,28 Pale Ale
8,28 Triple,28 Triple
9,44 Blanche,44 Blanche


In [ ]:
brands_df['beer_brand'].unique().size

2061

In [ ]:
brands_df['beer_brand_corrected'].unique().size

2061

In [ ]:
query = """
  MATCH (b:BeerBrand WHERE b.name = $beer_brand)
  SET b.name = $beer_brand_corrected
"""

In [ ]:
for row in brands_df.iterrows():
  parameters = {'beer_brand': row[1]['beer_brand'], 'beer_brand_corrected': row[1]['beer_brand_corrected']}
  app.query_params(query, parameters)

ERROR:neo4j.io:[#D48C]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('52.151.91.171', 7687)) (ResolvedIPv4Address(('52.151.91.171', 7687))): SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2426)')
ERROR:neo4j.pool:Unable to retrieve routing information


In [ ]:
query = """
  MATCH (b:BeerBrand)
  SET b:Beer
  REMOVE b:BeerBrand
"""

In [ ]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27e31b520>, keys=[])

### Correct brewery names

In [ ]:
query = """
  MATCH (b:Brewery)
  RETURN b.name
"""

In [ ]:
result = app.query(query)

In [ ]:
breweries = [r.values()[0] for r in result.records]
brewery_df = pd.DataFrame(data=breweries, columns=['brewery'])

In [ ]:
brewery_df.head()

,brewery
0,Brasserie Grain d'Orge
1,Brasserie de Jandrain-Jandrenouille
2,Brouwerij Anders! voor VZW De Winning Maatwerk
3,Brasserie L'Échappée Belle
4,De Proefbrouwerij voor Caulier Developpement


In [ ]:
brewery_df['brewery_corrected'] = brewery_df['brewery'].apply(lambda x: re.sub(r'[[0-9]*]', '', x))
brewery_df['brewery_corrected'] = brewery_df['brewery_corrected'].apply(lambda x: x.split(',')[0])
brewery_df['brewery_corrected'] = brewery_df['brewery_corrected'].apply(lambda x: re.sub(r'de La', 'de la', x))

In [ ]:
brewery_df

,brewery,brewery_corrected
0,Brasserie Grain d'Orge,Brasserie Grain d'Orge
1,Brasserie de Jandrain-Jandrenouille,Brasserie de Jandrain-Jandrenouille
2,Brouwerij Anders! voor VZW De Winning Maatwerk,Brouwerij Anders! voor VZW De Winning Maatwerk
3,Brasserie L'Échappée Belle,Brasserie L'Échappée Belle
4,De Proefbrouwerij voor Caulier Developpement,De Proefbrouwerij voor Caulier Developpement
...,...,...
494,De Proefbrouwerij voor Oude Brouwerij van Zonn...,De Proefbrouwerij voor Oude Brouwerij van Zonn...
495,De Proefbrouwerij voor de gemeente Zwalm,De Proefbrouwerij voor de gemeente Zwalm
496,Zennebrouwerij,Zennebrouwerij
497,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen


In [ ]:
brewery_df['brewery'].unique().size

492

In [ ]:
brewery_df['brewery_corrected'].unique().size

491

In [ ]:
query = """
  MATCH (b:Brewery WHERE b.name = $brewery)
  SET b.name = $brewery_corrected
"""

In [ ]:
for row in brewery_df.iterrows():
  parameters = {'brewery': row[1]['brewery'], 'brewery_corrected': row[1]['brewery_corrected']}
  app.query_params(query, parameters)

In [ ]:
query = """
  MATCH (b:Brewery)
  WITH b.name AS name, collect(b) as nodes
  WHERE size(nodes) > 1
  CALL apoc.refactor.mergeNodes(nodes)
  YIELD node
  RETURN node
"""

In [ ]:
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27ed0d0c0>, keys=['node'])

### Correct Beer Types

In [ ]:
query = """
  MATCH (n:BeerType)
  WITH n as beertype, split(n.name, ',') as splitted
  UNWIND splitted as split
  WITH beertype as beertype, ltrim(rtrim(lower(split))) as split
  MERGE (t:Type{type: split})
  CREATE (beertype)-[:HAS_TYPE]->(t)
"""

In [ ]:
app.query(query)

ERROR:neo4j.io:[#A270]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('52.151.91.171', 7687)) (ResolvedIPv4Address(('52.151.91.171', 7687))): OSError('No data')
ERROR:neo4j.pool:Unable to retrieve routing information


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27ebec9a0>, keys=[])

In [ ]:
query = """
  MATCH (bb:BeerBrand)-[:IS_A]-(bt:BeerType)-[:HAS_TYPE]-(t:Type)
  MERGE (bb)-[:IS_TYPE]->(t)
"""

In [ ]:
app.query(query)

In [ ]:
query = """
  MATCH (bt:BeerType) DETACH DELETE bt
"""

In [ ]:
app.query(query)

In [ ]:
b

### Set Constraints

In [ ]:
app.query("CREATE CONSTRAINT unique_beer IF NOT EXISTS FOR (b:Beer) REQUIRE b.name IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27e2ff220>, keys=[])

In [ ]:
app.query("CREATE CONSTRAINT unique_type IF NOT EXISTS FOR (b:Type) REQUIRE b.name IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27edaaad0>, keys=[])

In [ ]:
app.query("CREATE CONSTRAINT unique_brewery IF NOT EXISTS FOR (b:Brewery) REQUIRE b.name IS UNIQUE")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7ca27e2ff3d0>, keys=[])

## Create Pieces of Text

In [ ]:
model = 'gpt-3.5-turbo-0125'

In [ ]:
llm = ChatOpenAI(model=model, temperature=0)

In [ ]:
system_prompt = """
  You are a helpful assistant that provides more information on beers.
  Please provide a description on the specified beer.
  Focus on giving a description with ingredients, type of beer, flavour, taste etc. etc.
  Please don't mention the brewery in your answer.
  Please don't put any line breaks (\n) in your answer.
  """

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Please give me a description of the beer {beer} from brewery {brewery}."),
    ]
)

chain = prompt | llm

In [ ]:
def invoke_chain(chain, beer, brewery):
  result = chain.invoke(
      {
          "beer": beer,
          "brewery": brewery,
      }
    )
  return result.content

In [ ]:
invoke_chain(chain, "44 Blonde", "Brasserie de La Gleize")

'44 Blonde is a Belgian-style blonde ale that is brewed with a combination of Pilsner malt, wheat, and a touch of aromatic hops. This beer has a light golden color with a slightly hazy appearance and a fluffy white head. It offers a delicate balance of fruity esters and spicy phenols from the Belgian yeast strain, giving it a complex aroma with notes of citrus, banana, and clove. The flavor profile is crisp and refreshing, with a subtle sweetness from the malt and a gentle bitterness that lingers on the palate. 44 Blonde has a medium body and a moderate carbonation, making it easy to drink and perfect for enjoying on a warm day.'

In [ ]:
invoke_chain(chain, "44 Blonde", "Brasserie de La Gleize")


In [ ]:
query = """
  MATCH (beer:Beer)-[:BREWS]-(brew:Brewery)
  RETURN beer.name as beer, brew.name AS brewery
"""

In [ ]:
result = app.query(query)

In [ ]:
beers = [[r.values()[0], r.values()[1]] for r in result.records]
beers_df = pd.DataFrame(data=beers, columns=['beer', 'brewery'])

In [ ]:
beers_df

,beer,brewery
0,Joup,Brasserie Grain d'Orge
1,Canaille,Brasserie Grain d'Orge
2,Hervoise,Brasserie Grain d'Orge
3,Grelotte,Brasserie Grain d'Orge
4,Brice,Brasserie Grain d'Orge
...,...,...
2079,Zonnegemse Zot,De Proefbrouwerij voor Oude Brouwerij van Zonn...
2080,Zwalmse Tripel,De Proefbrouwerij voor de gemeente Zwalm
2081,Zwarte Piet,Zennebrouwerij
2082,Zwet.be,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen


In [ ]:
beers_df['description'] = beers_df.apply(lambda x: invoke_chain(chain, x.beer, x.brewery), axis=1)

In [ ]:
beers_df

,beer,brewery,description
0,Joup,Brasserie Grain d'Orge,Joup is a Belgian-style Saison beer brewed wit...
1,Canaille,Brasserie Grain d'Orge,Canaille is a Belgian Strong Pale Ale brewed w...
2,Hervoise,Brasserie Grain d'Orge,Hervoise is a Belgian-style wheat beer brewed ...
3,Grelotte,Brasserie Grain d'Orge,Grelotte is a Belgian-style beer brewed with a...
4,Brice,Brasserie Grain d'Orge,"I'm sorry, but I couldn't find specific inform..."
...,...,...,...
2079,Zonnegemse Zot,De Proefbrouwerij voor Oude Brouwerij van Zonn...,Zonnegemse Zot is a Belgian-style blonde ale b...
2080,Zwalmse Tripel,De Proefbrouwerij voor de gemeente Zwalm,Zwalmse Tripel is a Belgian-style tripel beer ...
2081,Zwarte Piet,Zennebrouwerij,Zwarte Piet is a traditional Belgian-style dar...
2082,Zwet.be,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,Zwet.be is a unique collaboration beer between...


In [ ]:
query = """
  MATCH (b:Beer WHERE b.name = $beer)
  SET b.description = $description
"""

In [ ]:
for row in beers_df.iterrows():
  parameters = {'beer': row[1]['beer'], 'description': row[1]['description']}
  app.query_params(query, parameters)

## KG Builder

In [ ]:
query = """
  MATCH (beer:Beer)-[]-(brew:Brewery)
  RETURN beer.name as name, brew.name as brewery, beer.description as description
"""

In [ ]:
result = app.query(query)

In [ ]:
beers = [r.values() for r in result.records]
beers_df = pd.DataFrame(data=beers, columns=['beer', 'brewery', 'description'])

In [ ]:
beers_df

,beer,brewery,description
0,Joup,Brasserie Grain d'Orge,Joup is a Belgian-style Saison beer brewed wit...
1,Canaille,Brasserie Grain d'Orge,Canaille is a Belgian Strong Pale Ale brewed w...
2,Hervoise,Brasserie Grain d'Orge,Hervoise is a Belgian-style wheat beer brewed ...
3,Grelotte,Brasserie Grain d'Orge,Grelotte is a Belgian-style beer brewed with a...
4,Brice,Brasserie Grain d'Orge,"I'm sorry, but I couldn't find specific inform..."
...,...,...,...
2079,Zonnegemse Zot,De Proefbrouwerij voor Oude Brouwerij van Zonn...,Zonnegemse Zot is a Belgian-style blonde ale b...
2080,Zwalmse Tripel,De Proefbrouwerij voor de gemeente Zwalm,Zwalmse Tripel is a Belgian-style tripel beer ...
2081,Zwarte Piet,Zennebrouwerij,Zwarte Piet is a traditional Belgian-style dar...
2082,Zwet.be,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,Zwet.be is a unique collaboration beer between...


In [ ]:
model = 'gpt-3.5-turbo-0125'

In [ ]:
llm = ChatOpenAI(model=model, temperature=0)

In [ ]:
system_prompt = """
  You are an assistant to extract entities from a piece of text.
  The input consists out of the beer name and a desciption. These descriptions contain text with more detail on the beer. They contains their type, flavour, ingredients etc.
  Please extract these entities from the piece of texts. Don't include any relationsihps or entities regarding breweries. These already exist.
  The response should be represented as a a Cypher relationship. In which the two nodes are merged first and then the relationship is created. Make sure that the created entities always have the label :Entity. The labels should be capatilized as: :Entity:Characteristic. Also, lower case the properties unless there are names in their. Please make the relationships singular. So HAS_CHARACTERISTIC instead of HAS_CHARACTERISTICS and IS_TYPE instead of IS_TYPES.
  Please don't include any new line in your answer (\n)

  Use the following structure to give an answer: MATCH (b:Beer{{name: "<beer_name>"}}) MERGE (e:Entity:<Label>{{name: "<property>"}}) MERGE (b)-[:<REL_TYPE>]-(e)

  Examples would be the following:
  Beer name: "44 Blonde"
  Beer description: "44 Blonde is a Belgian-style blonde ale that is brewed with a combination of Pilsner malt, wheat, and a touch of aromatic hops. This beer has a light golden color with a slightly hazy appearance and a fluffy white head. It offers a balanced flavor profile with notes of citrus, floral hops, and a subtle sweetness from the malt. The taste is crisp and refreshing, with a medium body and a clean, dry finish. 44 Blonde is a versatile beer that pairs well with a variety of dishes, making it a popular choice for those looking for a classic Belgian ale experience."
  Response: "MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Type{{name: "blonde ale"}}) MERGE (b)-[:IS_TYPE]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Ingredient{{name: "malt"}} MERGE (b)-[:HAS_INGREDIENT]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Ingredient{{name: "wheat"}} MERGE (b)-[:HAS_INGREDIENT]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Ingredient{{name: "aromatic hops"}}) MERGE (b)-[:HAS_INGREDIENT]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Color{{name: "light golden"}}) MERGE (b)-[:HAS_COLOR]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Appearance{{name: "fluffy white head"}}) MERGE (b)-[:APPEARS]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Appearance{{name: "slightly hazy"}}) MERGE (b)-[:APPEARS]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Flavor{{name: "citrus"}}) MERGE (b)-[:HAS_FLAVOR]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Flavor{{name: "floral hops"}}) MERGE (b)-[:HAS_FLAVOR]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Flavor{{name: "sweet"}}) MERGE (b)-[:HAS_FLAVOR]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Taste{{name: "crisp"}}) MERGE (b)-[:HAS_TASTE]-; MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Taste{{name: "refreshing"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Taste{{name: "medium body"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Taste{{name: "clean finish"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (Entity:Taste{{name: "dry finish"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (:Entity:CHARACTERISTIC{{name: "versatile"}}) MERGE (b)-[:HAS_CHARACTERISTIC]-(e); MATCH (b:Beer{{name: "44 Blonde"}}) MERGE (e:Entity:Taste{{name: "classic Belgian ale"}} MERGE (b)-[:IS_TYPE]-(e)"

  Beer name: "421 Pintje"
  Beer description: "421 Pintje is a Belgian Pale Ale brewed with a blend of Pilsner and Munich malts, giving it a balanced malt profile with a touch of sweetness. It is hopped with a combination of European hops, providing a floral and slightly spicy aroma. This beer has a medium body and a moderate bitterness level, making it easy to drink. The flavor profile includes notes of biscuit, caramel, and a hint of citrus from the hops. Overall, 421 Pintje is a refreshing and flavorful beer with a smooth finish.
  Response: " MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Type{{name: "pale ale"}}) MERGE (b)-[:IS_TYPE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Blend{{name: "pilsner"}}) MERGE (b)-[:HAS_BLEND]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Blend{{name: "Munich malts"}}) MERGE (b)-[:HAS_BLEND]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Profile{{name: "balanced malt"}}) MERGE (b)-[:HAS_PROFILE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (:Entity:Profile{{name: "sweet"}}) MERGE (b)-[:HAS_PROFILE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Ingredient{{name: "European hops"}}) MERGE (b)-[:HAS_INGREDIENT]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Aroma{{name: "floral"}}) MERGE (b)-[:HAS_AROMA]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Aroma{{name: "slighly spicy"}}) MERGE (b)-[:HAS_AROMA]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "medium body"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "moderate bitterness level"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "biscuit"}}) MERGE (b)-[:HAS_AROMA]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "caramel"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "citrus"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "refreshing"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) NERGE (e:Entity:Taste{{name: "flavorful"}}) MERGE (b)-[:HAS_TASTE]-(e); MATCH (b:Beer{{name: "421 Pintje"}}) MERGE (e:Entity:Taste{{name: "smooth finish"}}) MERGE (b)-[:HAS_TASTE]-(e)"

  If the description doesn't contain any specific information on the beer. Please provide an empty string as response: ''. No entities should be extracted.
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Please extract entities for the beer: {beer_name} with description: {beer_description}. "),
    ]
)

chain = prompt | llm

In [ ]:
def invoke_chain(chain, row):
  beer = row['beer']
  description = row['description']
  result = chain.invoke(
      {
          "beer_name": beer,
          "beer_description": description,
      }
    )
  return result.content

In [ ]:
beers_df

,beer,brewery,description
0,Joup,Brasserie Grain d'Orge,Joup is a Belgian-style Saison beer brewed wit...
1,Canaille,Brasserie Grain d'Orge,Canaille is a Belgian Strong Pale Ale brewed w...
2,Hervoise,Brasserie Grain d'Orge,Hervoise is a Belgian-style wheat beer brewed ...
3,Grelotte,Brasserie Grain d'Orge,Grelotte is a Belgian-style beer brewed with a...
4,Brice,Brasserie Grain d'Orge,"I'm sorry, but I couldn't find specific inform..."
...,...,...,...
2079,Zonnegemse Zot,De Proefbrouwerij voor Oude Brouwerij van Zonn...,Zonnegemse Zot is a Belgian-style blonde ale b...
2080,Zwalmse Tripel,De Proefbrouwerij voor de gemeente Zwalm,Zwalmse Tripel is a Belgian-style tripel beer ...
2081,Zwarte Piet,Zennebrouwerij,Zwarte Piet is a traditional Belgian-style dar...
2082,Zwet.be,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,Zwet.be is a unique collaboration beer between...


In [ ]:
# beer = beers_df.iloc[0]
# beer_name = beer.beer
# beer_description = beer.description
# result = invoke_chain(chain, beer_name, beer_description)

# result = invoke_chain(chain, beers_df.iloc[0])
# print(result)
# split_chars = [';', '\n\n']
# pattern = f"[{''.join(split_chars)}]"
# re.split(pattern, result)

In [ ]:
beers_df['entities'] = beers_df.apply(lambda x: invoke_chain(chain, x), axis=1)

In [ ]:
beers_df.to_csv('beers_df.csv')
!cp beers_df.csv "drive/MyDrive/Colab Notebooks/"

## Load entities to Graph

In [ ]:
beers_df = pd.read_csv("drive/MyDrive/Colab Notebooks/beers_df.csv")[['beer', 'brewery', 'description', 'entities']]

In [ ]:
#app.query("""MATCH (b:Beer{name: "Doiminicains Tripel"}) SET b.name = "Dominicains Tripel" """)
#app.query("""MATCH (b:Beer{name: "Cuvée de ciney"}) SET b.name = "Cuvée de Ciney" """)
#app.query("""MATCH (b:Beer{name: "Black Damnation VIII (S.H.I.T))"}) SET b.name = "Black Damnation VIII (S.H.I.T)" """)
#app.query("""MATCH (b:Beer{name: "Dikke Jan tripel"}) SET b.name = "Dikke Jan Tripel" """)
#app.query("""MATCH (b:Beer{name: "Maredsous tripel"}) SET b.name = "Maredsous Tripel" """)
#app.query("""MATCH (b:Beer{name: "Roeselaarse tripel"}) SET b.name = "Roeselaarse Tripel" """)
#app.query("""MATCH (b:Beer{name: "Dikke Jan quadruple"}) SET b.name = "Dikke Jan Quadruple" """)
#app.query("""MATCH (b:Beer{name: "Corsendonk Dubbel kriek"}) SET b.name = "Corsendonk Dubbel Kriek" """)
#app.query("""MATCH (b:Beer{name: "St. Louis Frambozen(bier)"}) SET b.name = "St. Louis Frambozen" """)
#app.query("""MATCH (b:Beer{name: "Vapeur cochonne"}) SET b.name = "Vapeur Cochonne" """)
#app.query("""MATCH (b:Beer) WHERE b.name STARTS WITH "'" SET b.name = ltrim(b.name, "'")   """)
#app.query("""MATCH (b:Beer) WHERE b.name CONTAINS '"' SET b.name = replace(b.name, '"', "'")   """)

In [ ]:
beers_df

,beer,brewery,description,entities
0,Joup,Brasserie Grain d'Orge,Joup is a Belgian-style Saison beer brewed wit...,"MATCH (b:Beer{name: ""Joup""}) MERGE (e:Entity:T..."
1,Canaille,Brasserie Grain d'Orge,Canaille is a Belgian Strong Pale Ale brewed w...,"MATCH (b:Beer{name: ""Canaille""}) MERGE (e:Enti..."
2,Hervoise,Brasserie Grain d'Orge,Hervoise is a Belgian-style wheat beer brewed ...,"MATCH (b:Beer{name: ""Hervoise""}) MERGE (e:Enti..."
3,Grelotte,Brasserie Grain d'Orge,Grelotte is a Belgian-style beer brewed with a...,"MATCH (b:Beer{name: ""Grelotte""}) MERGE (e:Enti..."
4,Brice,Brasserie Grain d'Orge,"I'm sorry, but I couldn't find specific inform...",''
...,...,...,...,...
2079,Zonnegemse Zot,De Proefbrouwerij voor Oude Brouwerij van Zonn...,Zonnegemse Zot is a Belgian-style blonde ale b...,"MATCH (b:Beer{name: ""Zonnegemse Zot""}) MERGE (..."
2080,Zwalmse Tripel,De Proefbrouwerij voor de gemeente Zwalm,Zwalmse Tripel is a Belgian-style tripel beer ...,"MATCH (b:Beer{name: ""Zwalmse Tripel""}) MERGE (..."
2081,Zwarte Piet,Zennebrouwerij,Zwarte Piet is a traditional Belgian-style dar...,"MATCH (b:Beer{name: ""Zwarte Piet""}) MERGE (e:E..."
2082,Zwet.be,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,Zwet.be is a unique collaboration beer between...,"MATCH (b:Beer{name: ""Zwet.be""}) MERGE (e:Entit..."


In [ ]:
split_chars = [';', '\n\n']
pattern = f"[{''.join(split_chars)}]"

re.split(pattern, beers_df['entities'].iloc[120])

['MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Type{name: "strong dark ale"}) MERGE (b)-[:IS_TYPE]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Ingredient{name: "caramel malts"}) MERGE (b)-[:HAS_INGREDIENT]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Ingredient{name: "roasted malts"}) MERGE (b)-[:HAS_INGREDIENT]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Flavor{name: "rich"}) MERGE (b)-[:HAS_FLAVOR]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Flavor{name: "complex"}) MERGE (b)-[:HAS_FLAVOR]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Ingredient{name: "Belgian candi sugar"}) MERGE (b)-[:HAS_INGREDIENT]-(e)',
 ' MATCH (b:Beer{name: "Abbaye de Saint-Martin Cuvée de Noël"}) MERGE (e:Entity:Spice{name: "cinnamon"}) MERGE (b)-[:HAS_SPICE]-(e)',
 ' MATCH 

In [ ]:
for row in beers_df[1902:1903].iterrows():
  beer_name = row[1]['beer']
  beer_description = row[1]['description']
  entities = row[1]['entities']

  split_chars = [';', '\n\n']
  pattern = f"[{''.join(split_chars)}]"

  for r in re.split(pattern, entities):
    if (r != '') & (r != "''"):
      if r[-1] == '"':
        r = r[:-1]
      app.query(r)

In [ ]:
app.query("""MATCH (n:CHARACTERISTIC)
  SET n:Characteristic
  REMOVE n:CHARACTERISTIC"""
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x78b75fef4eb0>, keys=[])

In [ ]:
app.query("""MATCH (n:Characterisitc)
  SET n:Characteristic
  REMOVE n:Characterisitc"""
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x78b75fef7460>, keys=[])

## Brewery Locations

In [ ]:
query = """
  MATCH (b:Brewery)
  RETURN DISTINCT b.name as brewery
"""

In [ ]:
result = app.query(query)

In [ ]:
breweries = [r.values() for r in result.records]
breweries_df = pd.DataFrame(data=breweries, columns=['brewery'])

In [ ]:
model = 'gpt-4o'

In [ ]:
llm = ChatOpenAI(model=model, temperature=0)

In [ ]:
system_prompt = """
  You are an assistant to find addresses of beer breweries in Belgium.
  Please based on the name of the brewery: give the address, city, latitude and longitude).
  Please give this as a python dictionary. If you cannot find the address just return an empty dictionary. Please don't include any new lines in your answer.
  Please use double quotes for both keys as items in the dictionary. Example: dict = {{"key": "item"}}.
  If needed please do a google search to find the address.
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Please find the address, city, latitude and longitude of Belgian brewery: {brewery}."),
    ]
)

chain = prompt | llm

In [ ]:
def invoke_chain(chain, row):
  brewery = row['brewery']
  result = chain.invoke(
      {
          "brewery": brewery,
      }
    )
  return result.content

In [ ]:
breweries_df

,brewery,address,dict,street,city,latitude,longitude
0,Brasserie Grain d'Orge,"{""address"": ""4 Rue de la Gare"", ""city"": ""Hombo...","{'address': '4 Rue de la Gare', 'city': 'Hombo...",4 Rue de la Gare,Hombourg,50.7261,5.9197
1,Brasserie de Jandrain-Jandrenouille,"{""address"": ""Rue de la Brasserie 4"", ""city"": ""...","{'address': 'Rue de la Brasserie 4', 'city': '...",Rue de la Brasserie 4,Jandrain-Jandrenouille,50.6833,4.9833
2,Brouwerij Anders! voor VZW De Winning Maatwerk,"{""address"": ""Industrielaan 28"", ""city"": ""Halen...","{'address': 'Industrielaan 28', 'city': 'Halen...",Industrielaan 28,Halen,50.9405,5.1107
3,Brasserie L'Échappée Belle,"{""address"": ""1 Rue de la Gare"", ""city"": ""L'Isl...","{'address': '1 Rue de la Gare', 'city': 'L'Isl...",1 Rue de la Gare,L'Isle-Jourdain,43.6153,1.0806
4,De Proefbrouwerij voor Caulier Developpement,"{""address"": ""Doornzelestraat 114"", ""city"": ""Lo...","{'address': 'Doornzelestraat 114', 'city': 'Lo...",Doornzelestraat 114,Lochristi,51.0961,3.8375
...,...,...,...,...,...,...,...
486,De Proefbrouwerij voor Oude Brouwerij van Zonn...,"{""address"": ""Hijfte 91"", ""city"": ""Lochristi"", ...","{'address': 'Hijfte 91', 'city': 'Lochristi', ...",Hijfte 91,Lochristi,51.0961,3.8375
487,De Proefbrouwerij voor de gemeente Zwalm,"{""address"": ""Doornzelestraat 114"", ""city"": ""Lo...","{'address': 'Doornzelestraat 114', 'city': 'Lo...",Doornzelestraat 114,Lochristi,51.0801,3.8373
488,Zennebrouwerij,"{""address"": ""Delaunoystraat 58"", ""city"": ""Brus...","{'address': 'Delaunoystraat 58', 'city': 'Brus...",Delaunoystraat 58,Brussels,50.8609,4.3314
489,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,"{""address"": ""Dennenlaan 9"", ""city"": ""Lochristi...","{'address': 'Dennenlaan 9', 'city': 'Lochristi...",Dennenlaan 9,Lochristi,51.0961,3.8373


In [ ]:
breweries_df['address'] = breweries_df.apply(lambda x: invoke_chain(chain, x), axis=1)
breweries_df['dict'] = breweries_df['address'].apply(lambda x: json.loads(x))
breweries_df['street'] = breweries_df['dict'].apply(lambda x: x['address'] if 'address' in x.keys() else None)
breweries_df['city'] = breweries_df['dict'].apply(lambda x: x['city'] if 'city' in x.keys() else None)
breweries_df['latitude'] = breweries_df['dict'].apply(lambda x: x['latitude'] if 'latitude' in x.keys() else None)
breweries_df['longitude'] = breweries_df['dict'].apply(lambda x: x['longitude'] if 'longitude' in x.keys() else None)

In [ ]:
breweries_df

,brewery,address,dict,street,city,latitude,longitude
0,Brasserie Grain d'Orge,"{""address"": ""Rue d'Aubel 1"", ""city"": ""Hombourg...","{'address': 'Rue d'Aubel 1', 'city': 'Hombourg...",Rue d'Aubel 1,Hombourg,50.7011,5.9193
1,Brasserie de Jandrain-Jandrenouille,"{""address"": ""Rue de la Brasserie 4"", ""city"": ""...","{'address': 'Rue de la Brasserie 4', 'city': '...",Rue de la Brasserie 4,Jandrain-Jandrenouille,50.6683,4.9575
2,Brouwerij Anders! voor VZW De Winning Maatwerk,"{""address"": ""Industrielaan 4"", ""city"": ""Halen""...","{'address': 'Industrielaan 4', 'city': 'Halen'...",Industrielaan 4,Halen,50.9406,5.1103
3,Brasserie L'Échappée Belle,"{""address"": ""Rue de la Station 38"", ""city"": ""F...","{'address': 'Rue de la Station 38', 'city': 'F...",Rue de la Station 38,Froidchapelle,50.1525,4.3331
4,De Proefbrouwerij voor Caulier Developpement,"{""address"": ""Doornzelestraat 114"", ""city"": ""Lo...","{'address': 'Doornzelestraat 114', 'city': 'Lo...",Doornzelestraat 114,Lochristi,51.0961,3.8375
...,...,...,...,...,...,...,...
486,De Proefbrouwerij voor Oude Brouwerij van Zonn...,"{""address"": ""Hellestraat 24"", ""city"": ""Lochris...","{'address': 'Hellestraat 24', 'city': 'Lochris...",Hellestraat 24,Lochristi,51.0806,3.8375
487,De Proefbrouwerij voor de gemeente Zwalm,"{""address"": ""Doornzelestraat 114"", ""city"": ""Lo...","{'address': 'Doornzelestraat 114', 'city': 'Lo...",Doornzelestraat 114,Lochristi,51.0806,3.8375
488,Zennebrouwerij,"{""address"": ""Gentsesteenweg 556"", ""city"": ""And...","{'address': 'Gentsesteenweg 556', 'city': 'And...",Gentsesteenweg 556,Anderlecht,50.8356,4.3033
489,De Proefbrouwerij voor Geuzestekerij 3 Fonteinen,"{""address"": ""Molenstraat 47"", ""city"": ""Beersel...","{'address': 'Molenstraat 47', 'city': 'Beersel...",Molenstraat 47,Beersel,50.7675,4.2953


In [ ]:
breweries_df = breweries_df.fillna('NULL')

In [ ]:
for row in breweries_df.iterrows():
  brewery = row[1]['brewery']
  street = row[1]['street']
  city = row[1]['city']
  latitude = row[1]['latitude']
  longitude = row[1]['longitude']

  query = f"""
    MATCH (b:Brewery)
    WHERE b.name = "{brewery}"
    SET b.street = "{street}"
    SET b.city = "{city}"
    SET b.latitude = {latitude}
    SET b.longitude = {longitude}
  """
  app.query(query)

In [ ]:
query = """
MATCH (b:Brewery)
SET b.point = point({latitude: b.latitude, longitude: b.longitude})
"""
app.query(query)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7c15237ed3c0>, keys=[])

In [ ]:
query ="""
    CALL apoc.periodic.iterate(
        "MATCH (e1:Entity) RETURN e1",
        "WITH e1 MATCH (e2:Entity) WHERE e1.name = e2.name WITH COLLECT(e1) + COLLECT(e2) as nodes CALL apoc.refactor.mergeNodes(nodes,{properties:'combine', mergeRels:true}) YIELD node RETURN count(*)",
        {batchSize: 1000}
    )
"""

In [ ]:
app.query(query)

EagerResult(records=[<Record batches=5 total=4160 timeTaken=3 committedOperations=1000 failedOperations=0 failedBatches=4 retries=0 errorMessages={} batch={'total': 5, 'errors': {'org.neo4j.graphdb.NotFoundException: Node 4:51e982c7-f6e6-4cac-9bcd-7098b952059b:5587 not found.': 1, 'org.neo4j.graphdb.NotFoundException: Node 4:51e982c7-f6e6-4cac-9bcd-7098b952059b:4578 not found.': 1, 'org.neo4j.graphdb.NotFoundException: Node 4:51e982c7-f6e6-4cac-9bcd-7098b952059b:6595 not found.': 1, 'org.neo4j.graphdb.NotFoundException: Node 4:51e982c7-f6e6-4cac-9bcd-7098b952059b:7584 not found.': 1}, 'committed': 1, 'failed': 4} operations={'total': 4160, 'errors': {}, 'committed': 1000, 'failed': 0} wasTerminated=False failedParams={} updateStatistics={'relationshipsDeleted': 0, 'relationshipsCreated': 0, 'nodesDeleted': 0, 'nodesCreated': 0, 'labelsRemoved': 0, 'labelsAdded': 0, 'propertiesSet': 0}>], summary=<neo4j._work.summary.ResultSummary object at 0x7bab57d0ff40>, keys=['batches', 'total', 'ti